In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as A
import pandas as pd
from sklearn.manifold import TSNE
import sklearn.manifold 
from matplotlib import pyplot as plt
import re
import codecs
from numpy  import array
from scipy import stats
# from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import os
from tensorboardX import SummaryWriter

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# PREPROCESSING

In [2]:
from preprocessing import Preprocess
prep  = Preprocess()

In [3]:
def get_input_layer(word_idx):
    x = torch.zeros(prep.vocabulary_size).float()
    x[word_idx] = 1.0
    return x

# this is a one hot encoded something

# TRAINING

In [4]:
# testWriter = SummaryWriter()

In [5]:
def stop_training():
    change = priorLoss / avgLoss.item()
    if (priorLoss == 0 or change > 1.05):
        return False
    else:
        return True

In [6]:
dtype = torch.float 
dimensionSize = 4
num_epochs = 2
learning_rate = 0.001
priorLoss = 0
avgLoss = 0

firstLayer = torch.randn(dimensionSize, prep.vocabulary_size, requires_grad=True).float()
secondLayer = torch.randn(prep.vocabulary_size, dimensionSize, requires_grad=True).float()

for epo in range(num_epochs):
    loss_val = 0
    for data, target in prep.idx_pairs:

        # one hot encoded tensor
        x = get_input_layer(data).float()

        # target word 
        y_true = torch.from_numpy(np.array([target])).long()

        # Hidden Layer: gradient magic happening ...
        z1 = torch.matmul(firstLayer, x)
        z2 = torch.matmul(secondLayer, z1)

        # introducing non-linearity
        softmax = A.LogSoftmax()
        soft_max_output = softmax(z2)
        # neg_log_likelihood
        loss = F.nll_loss(soft_max_output.view(1,-1), y_true)
        loss_val += loss  # this might be to please the  framework, and adding stuff to the gradient calculator
        
        # propagating it back
        loss.backward()
        
        # updating the weights of both layers
        firstLayer.data -= learning_rate * firstLayer.grad.data
        secondLayer.data -= learning_rate * secondLayer.grad.data

        # set the gradients to zero for next iteration
        firstLayer.grad.data.zero_()
        secondLayer.grad.data.zero_()
    avgLoss = loss_val/len(prep.idx_pairs)
    print(f'Loss at epo {epo}: {avgLoss}')
#     testWriter.add_scalar('lossvalue', avgLoss, epo)
    if stop_training():
        break
    else:
        priorLoss = avgLoss.item()

#export scalar data to JSON for external processing
# writer.export_scalars_to_json("./all_scalars.json")
# testWriter.close

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Loss at epo 0: 5.876072406768799
Loss at epo 1: 4.792679309844971


In [7]:
F.nll_loss(soft_max_output.view(1,-1), y_true)

tensor(0.0604, grad_fn=<NllLossBackward>)

In [8]:
if not os.path.exists("SOFTMAX_trained"):
    os.makedirs("SOFTMAX_trained")

In [9]:
torch.save(secondLayer, os.path.join("SOFTMAX_trained", "test#2.w2v"))

# EVALUATING TRAINED MODEL

In [ ]:
loadedLayer = torch.load(os.path.join("SOFTMAX_trained", "5percent#1.w2v"))

In [ ]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# extract Wijeratne's Cosine_Similarities of the model which was trained on Google_Sense_Labels
google_sense_labels_score_508 = list(array(re.findall('(?<=Google_Sense_Label": )(.*?)(?=\,)', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
google_sense_labels = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = cosine_similarity(loadedLayer.detach().cpu().numpy()[word2idx[emoji1]].reshape(-1,300), loadedLayer.detach().cpu().numpy()[word2idx[emoji2]].reshape(-1,300))[0][0]
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        goldstandard.append(annotator_similarity_score_508.pop(0))
        selftrained.append(cosineSimilarity)
        google_sense_labels.append(float(google_sense_labels_score_508.pop(0)))
        

# skalierter GoldStandard
min_max_scaler = preprocessing.MinMaxScaler()
scaled_goldstandard = min_max_scaler.fit_transform(np.asarray(goldstandard).reshape(-1, 1))

print()

# computation of SPEARRANK CORRELATION COEFFICIENT
meinSPEARMAN = stats.spearmanr(goldstandard, selftrained)
seinSPEARMAN = stats.spearmanr(goldstandard, google_sense_labels)
print('mein Spearman: {}'.format(meinSPEARMAN.correlation))
print('sein Spearman: {}'.format(seinSPEARMAN.correlation))


# computation of MAE
meinMAE = mean_absolute_error(scaled_goldstandard, min_max_scaler.fit_transform(np.asarray(selftrained).reshape(-1, 1)))
seinMAE = mean_absolute_error(scaled_goldstandard, google_sense_labels)
print('mein MAE ist {}'.format(meinMAE))
print('sein MAE ist {}'.format(seinMAE))


# computation of MSE
meinMSE = mean_squared_error(scaled_goldstandard, min_max_scaler.fit_transform(np.asarray(selftrained).reshape(-1, 1)))
seinMSE = mean_squared_error(scaled_goldstandard, google_sense_labels)
print('mein MSE ist {}'.format(meinMSE))
print('sein MSE ist {}'.format(seinMSE))


In [ ]:
input = torch.randn(3, 5, requires_grad=True)
>>> # each element in target has to have 0 <= value < C
>>> target = torch.tensor([1, 0, 4])
>>> output = F.nll_loss(F.log_softmax(input), target)
>>> output.backward()

In [ ]:
output.size()

In [ ]:
tensor1 = torch.randn(1200, 5)
tensor2 = torch.randn(5, 80)
torch.matmul(tensor1, tensor2).size()

In [ ]:
tensor1 = torch.randn(2, 2)
print(tensor1)
print(torch.t(tensor1))
# tensor1.size()